<a href="https://colab.research.google.com/github/aee4/MedGemma/blob/main/scripts/Finetuning_with_text_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/medgemma-4b-pt"  # pretrained MedGemma
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,

    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from google.colab import files
uploaded = files.upload()



Saving eczema_dataset(main).csv to eczema_dataset(main) (1).csv


In [7]:
from datasets import load_dataset

# Load your dataset directly from CSV
dataset = load_dataset("csv", data_files="eczema_dataset(main) (1).csv", split="train")

# Split into train/test sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Display sample rows
print(train_dataset[:5])


Generating train split: 0 examples [00:00, ? examples/s]

{'disease': ['eczema', 'eczema', 'eczema', 'eczema', 'eczema'], 'text': ['On the wrists and ankles show faintly discolored patches that are darker or lighter than surrounding skin; patient notes they feels restless and loses sleep due to the itch. This is typically severe and persistent and can worsen during periods of hormonal change.', 'Symptoms include faintly discolored patches that are darker or lighter than surrounding skin at in skin creases and folds. The individual feels the skin is tight and raw after scratching — the eczema is chronic, requiring ongoing moisturization and care and linked to exposure to allergens like dust and pollen.', 'On the trunk and limbs show areas of skin that are tender and sore; patient notes they reports constant discomfort and burning sensation. This is typically chronic, requiring ongoing moisturization and care and sometimes flares after infections or insect bites.', 'Presentation: intensely itchy, dry and scaly patches of skin on on the trunk an

In [7]:
!pip uninstall -y bitsandbytes transformers accelerate
!pip install -U bitsandbytes accelerate transformers


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 131.1 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "google/medgemma-4b-pt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    dtype=torch.float16
)




The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!pip install datasets transformers accelerate -q


In [7]:
!pip install -q bitsandbytes transformers peft datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.0 MB/s eta 0:00:00


In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                      # rank
    lora_alpha=16,            # scaling factor
    target_modules=["q_proj", "v_proj"],  # LoRA layers to adapt (key for memory)
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,223,552 || all params: 4,303,303,024 || trainable%: 0.0749


In [10]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# Load dataset
df = pd.read_csv("eczema_dataset(main) (1).csv")

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Load tokenizer
model_name = "google/medgemma-4b-pt"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 🧠 Preprocessing function (with padding + truncation fix)
def preprocess_function(examples):
    texts = [
        f"disease: {d}\ntext: {t}"
        for d, t in zip(examples["disease"], examples["text"])
    ]
    tokenized = tokenizer(
        texts,
        max_length=512,
        truncation=True,
        padding="max_length",  # ✅ ensures consistent lengths
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 🧩 Apply preprocessing with batching
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# ✅ Ready for LoRA fine-tuning step
print("✅ Dataset tokenized successfully and ready for fine-tuning.")


Map:   0%|          | 0/699 [00:00<?, ? examples/s]

✅ Dataset tokenized successfully and ready for fine-tuning.


In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./medgemma-eczema",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)


/tmp/ipython-input-179496998.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()



The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eeyramagbetor (eeyramagbetor-taskarc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.000000
100,0.000000
150,0.000000
200,0.000000
250,0.000000


TrainOutput(global_step=264, training_loss=0.0, metrics={'train_runtime': 1541.2137, 'train_samples_per_second': 1.361, 'train_steps_per_second': 0.171, 'total_flos': 2.336721439747277e+16, 'train_loss': 0.0, 'epoch': 3.0})

In [14]:
model.save_pretrained("./aee4-lora-medgemma-eczema")


In [15]:
!ls -lh /content/aee4-lora-medgemma-eczema

total 13M
-rw-r--r-- 1 root root  855 Oct 29 09:16 adapter_config.json
-rw-r--r-- 1 root root  13M Oct 29 09:16 adapter_model.safetensors
-rw-r--r-- 1 root root 5.1K Oct 29 09:16 README.md
